# Paillier Homomorphic Encryption Example

DISCLAIMER: This is a proof-of-concept implementation. It does not represent a remotely product ready implementation or follow proper conventions for security, convenience, or scalability. It is part of a broader proof-of-concept demonstrating the vision of the OpenMined project, its major moving parts, and how they might work together.



## Introduction

This tutorial includes a basic example of the **Paillier Encryption Algorithm**, a homomorphic crypto system based on the fact that for a composite number $n$ and an integer $z$ it is hard to decide whether exists an $y$ such that $z \equiv y^n \pmod{n²}$. This is known as the *decisional composite residuosity assumption*.

This algorithm works as follows:

### Key generation:
(Taking the assumption that the primes chosen have the same length.)
1. Two large primes $p$ and $q$ are chosen randomly and independently of each other such that $\gcd(pq, (p-1)(q-1)) = 1$ (this assures that both have equal length).
2. Compute $n = pq$ and $\lambda = \phi(n)$, with $\phi(n) = (p-1)(q-1)$.
3. Assign $g = n+1$.
4. Set $\mu = \phi(n)^{-1} \pmod{n}$.

Now we have the following key pairs:
* The **public key** (to encrypt) is $(n, g)$,
* the **private or secret key** (to decrypt) is $(\lambda, \mu)$.

### Encryption:
1. $m$ has to be a message such that $0\leq m < n$.
2. Select a random $r$ with $0\leq r \leq n$.
3. The cyphertext $c$ is obtained as $c = g^m r^n \pmod{n²}$.

### Decryption:
If $c$ is the cyphertext, the plaintext message $m$ is obtained as 

$m = L(c^\lambda\pmod{n²})\cdot\mu \pmod{n}$, where $L(u) = (u-1)/n$.

## Dependencies

This example makes use of [KeyPair](https://github.com/OpenMined/PySyft/blob/261fdb10484b7ee586983f6dc74ad168d8d2f160/syft/he/paillier/keys.py#L122) and [PaillierTensor](https://github.com/OpenMined/PySyft/blob/261fdb10484b7ee586983f6dc74ad168d8d2f160/syft/he/paillier/basic.py#L5) classes to apply Paillier encryption, and [TensorBase](https://github.com/OpenMined/PySyft/blob/master/syft/tensor.py) (OpenMined's most generic API) to create the plaintext tensors that will be encrypted. [Numpy](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html) is just a package to manage N-dimensional array objects from which TensorBase is backed by.

In [1]:
from syft.he.paillier import KeyPair, PaillierTensor
from syft import TensorBase
import numpy as np

## Basic Operations

In order to encrypt data using Paillier encryption we need to generate a pair of public and private keys:

In [2]:
pubkey,prikey = KeyPair().generate()

Then we can create a numpy array the will be turned into an encrypted tensor using the public key (#1) and display it as a Python dictionary (#2):

In [3]:
x = PaillierTensor(pubkey, np.array([1, 2, 3, 4, 5.])) #1
x.data[0].__dict__ #2

{'_EncryptedNumber__ciphertext': 9587711831362355384080176835076503391887402166603761637772234557790650075228392822047892199736666809514467678786349341275460938847810308642534694688978494132896577572336276617154299744215160903522503690099880741370552392893182608428037769662168609940818575362994755362832863275425895218565617795585942090493122010042779337635207508633206881533543091702933151620686569817592620562358868621376259954072164780146749639389556194449875163103081128204966650842489381090836793094238206625151845499124220672507472840195823323499986572058793364898950831313510312858641586138283171063252628289194406732285088921814754960290337,
 '_EncryptedNumber__is_obfuscated': True,
 'exponent': -13,
 'public_key': <PaillierPublicKey f341d38459>}

We can now decrypt our tensor with the private key:

In [4]:
x.decrypt(prikey)

BaseTensor: array([ 1.,  2.,  3.,  4.,  5.])

Or decypt the result of adding the first element to the last one:

In [5]:
(x+x[0]).decrypt(prikey)

BaseTensor: array([ 2.,  3.,  4.,  5.,  6.])

We can also multiply it by a number:

In [6]:
(x*5).decrypt(prikey)

BaseTensor: array([  5.,  10.,  15.,  20.,  25.])

Or apply other linear (product, division, addition and difference) operations:

In [7]:
(x+x/5).decrypt(prikey)

BaseTensor: array([ 1.2,  2.4,  3.6,  4.8,  6. ])

## Key Serialization & Deserialization

Now we are going to see how can we serialize the keys once we generate them.
First we generate other pair of keys and encrypt our information with the public key:

In [8]:
pubkey,prikey = KeyPair().generate()

In [9]:
x = PaillierTensor(pubkey, np.array([1, 2, 3, 4, 5.]))

We can serialize our keys as follows with the `serialize` method for the [public](https://github.com/OpenMined/PySyft/blob/a0134fb5f6907d78a10b91427a4684d2e8279865/syft/he/paillier/keys.py#L110) and the [private](https://github.com/OpenMined/PySyft/blob/a0134fb5f6907d78a10b91427a4684d2e8279865/syft/he/paillier/keys.py#L48) keys respectively:

In [10]:
pubkey_str = pubkey.serialize()
pubkey_str

'{"public_key": {"n": 109039055901531821571085017615660123799215584968258690506185191484452435314904051295534637729119074988816164985557717406603192200148209057917286541848425374911845886779232233702260816343359797765293305330075997934933359460751097629954407220146335402048611025846625830604685135153097442084014462174614485842769}}'

In [11]:
prikey_str = prikey.serialize()
prikey_str

'{"secret_key": {"n": 109039055901531821571085017615660123799215584968258690506185191484452435314904051295534637729119074988816164985557717406603192200148209057917286541848425374911845886779232233702260816343359797765293305330075997934933359460751097629954407220146335402048611025846625830604685135153097442084014462174614485842769, "q": 12450639974350670618551941962659924758238938196620832453826560246908193468826211315992147874188293298282763970159173528504845385137696742812465693423293589, "p": 8757706923191187848191243449828007622575907000795615377147420049807395761942774441178453426172800445093730896202181604613820970343488909466080360137916621}}'

Then we can deserialize them and decrypt the information with the deserialized private (secret) key:

In [12]:
pubkey2,prikey2 = KeyPair().deserialize(pubkey_str,prikey_str)

In [13]:
prikey2.decrypt(x)

BaseTensor: array([ 1.,  2.,  3.,  4.,  5.])

In [14]:
y = PaillierTensor(pubkey,(np.ones(5))/2)

In [15]:
prikey.decrypt(y)

BaseTensor: array([ 0.5,  0.5,  0.5,  0.5,  0.5])

## Value Serialization & Deserialization

To serialize our tensors for a correct storage and transmission we use `pickle`.

In [16]:
import pickle

We can save it to a file path or by assigning it to an object through `dumps`:

In [17]:
y_str = pickle.dumps(y)

And convert it back to its original type with `loads`:

In [18]:
y2 = pickle.loads(y_str)

As we can see this assures that a decryption of the data can be done yet:

In [19]:
prikey.decrypt(y2)

BaseTensor: array([ 0.5,  0.5,  0.5,  0.5,  0.5])

## Resources

- [PySyft's Relevant Literature](https://github.com/OpenMined/PySyft/tree/a0134fb5f6907d78a10b91427a4684d2e8279865#relevant-literature)
- [PySyft's Homomorphic Encryption 101](https://github.com/OpenMined/Docs/blob/2029c26e6a35feed1b800e7423822037792e61e4/tutorials/homomorphic_crypto_101.md#homomorphic-encryption-101)